<a href="https://colab.research.google.com/github/Rishika70/Hack_To_Hire/blob/main/Hack_To_Hire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers[torch] tokenizers datasets evaluate rouge_score sentencepiece huggingface_hub --upgrade

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_c

In [2]:
!pip install bleu

  Preparing metadata (setup.py) ... done
  Created wheel for bleu: filename=bleu-0.3-py3-none-any.whl size=5780 sha256=eb364dcac4eb3bd0be9d055326dff1bb1d0b127d54b001971e42d502d452cf34
  Stored in directory: /root/.cache/pip/wheels/c6/d8/d1/009abe01b8b2c6a14c62d197b510b3cc1076014c22d712c5ce
Successfully built bleu


In [3]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertTokenizer, BertModel
from transformers import T5Tokenizer, T5Model
from transformers import GPT2Tokenizer, GPT2Model
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split


In [6]:
!pip install datasets
#Import the datasets library
from datasets import load_dataset


# Load and split the dataset
dataset = load_dataset("toughdata/quora-question-answer-dataset")
dataset = dataset["train"].train_test_split(test_size=0.2)
train_dataset = dataset["train"]
test_dataset = dataset["test"]



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/56402 [00:00<?, ? examples/s]

In [8]:

def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token.lower() not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

# Apply preprocessing to the 'question' and 'answer' columns
dataset = dataset.map(lambda x: {'question': preprocess_text(x['question']),
                                 'answer': preprocess_text(x['answer'])})

# Convert dataset to pandas DataFrame for splitting
df = dataset['train'].to_pandas()

# Split the data into training and validation sets
train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)

print("Training Data:")
print(train_data)
print("\nValidation Data:")
print(val_data)


Map:   0%|          | 0/45121 [00:00<?, ? examples/s]

Map:   0%|          | 0/11281 [00:00<?, ? examples/s]

Training Data:
                                                question  \
17394                      Whats # 1 thing always pray ?   
29099                          make final year project ?   
42067  think President Biden 's revocation Keystone X...   
24706  soldier pick fallen enemy weapon use ammo weap...   
35644          Bangladesh powerful prosperous Pakistan ?   
...                                                  ...   
11284                         Putin care people Russia ?   
44732                        's proper way eat sashimi ?   
38158  fairly basic understanding Python . want learn...   
860                http : //www.quora.com/profile/Me ...   
15795  Heard lot guru say go deep go inside . go ? go...   

                                                  answer  
17394  One famous successful business maneuver recent...  
29099  Explaining experience . start project idea res...  
42067  Typical . democrat ’ give damn Americans . dem...  
24706  ’ try give short answ

In [39]:
# Convert dataset to pandas DataFrame
df = pd.DataFrame(dataset['train'])



In [9]:
# To display the entirety of each row in the Dataset
pd.set_option('display.max_colwidth', None)

In [40]:
# Display first few rows of the dataset
print(df.head())

                                                                question  \
0              affordable web-based small business accounting software ?   
1                    right Sweden enter NATO without Russia 's consent ?   
2                              people still wear Speedo swimsuit beach ?   
3  gun fanatic say fact 130 study proving gun control work ? deny fact ?   
4                                        dog ever act offended him/her ?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

Dataset Rows & Columns count

In [11]:
# Dataset Rows & Columns count
df.shape

(45121, 2)

Dataset Information

In [12]:
# Dataset Info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45121 entries, 0 to 45120
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  45121 non-null  object
 1   answer    45121 non-null  object
dtypes: object(2)
memory usage: 705.1+ KB


Duplicate Values

In [13]:
# Dataset Duplicate Value Count
df.duplicated().sum()

904

Missing Values/Null Values

In [14]:
# Missing Values/Null Values Count
df.isna().sum()

question    0
answer      0
dtype: int64

Understanding Your Variables

In [15]:
# Dataset Columns
df.columns

Index(['question', 'answer'], dtype='object')

In [17]:
# Print the available columns in the DataFrame
print(df.columns)

Index(['question', 'answer'], dtype='object')


#Data Wrangling code

Clean the text


In [42]:

import re

def clean_text(text):
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    return text

df['question_cleaned'] = df['question'].apply(clean_text)
df['answer_cleaned'] = df['answer'].apply(clean_text)

print(df[['question_cleaned', 'answer_cleaned']])

                                                                                               question_cleaned  \
0                                                       affordable webbased small business accounting software    
1                                                             right sweden enter nato without russia s consent    
2                                                                      people still wear speedo swimsuit beach    
3                                           gun fanatic say fact 130 study proving gun control work  deny fact    
4                                                                                 dog ever act offended himher    
...                                                                                                         ...   
45116                                                                                    hiking gear never buy    
45117                                                                politely te

In [35]:
import pandas as pd

# Assuming _df_0 is your DataFrame
data = [
    {
        "question": "Affordable web-based small business accounting software?",
        "answer": ("Hi, software simple & easy use. Profit-oriented accounting software. Analyze budget upcoming financial year. "
                   "Also view past data within seconds. Free trial. Sign free trial: [GST Invoicing, Accounting & Filling Software](http://goo.gl/YA5QjJ) "
                   "Best accounting software also easy use. Try! Thanks.")
    },
    {
        "question": "Can Sweden enter NATO without Russia's consent?",
        "answer": ("Last knowledge update September 2021, Sweden is not a member of NATO (North Atlantic Treaty Organization). Therefore, it can enter NATO without Russia's consent. "
                   "NATO is a political and military alliance of 30 member countries in North America and Europe. Countries seeking to join NATO must go through a process of accession, "
                   "which involves negotiations, meeting certain criteria, and obtaining approval from existing NATO members. Sweden's foreign policy decisions are made by its government "
                   "based on national interests, security considerations, and diplomatic relations. Whether a country seeks to join a military alliance like NATO is a sovereign decision "
                   "typically based on assessments of security and regional stability. For developments regarding Sweden's NATO membership since September 2021, it is recommended to check "
                   "reliable, up-to-date sources for accurate information on the matter.")
    },
    {
        "question": "Do people still wear Speedo swimsuits at the beach?",
        "answer": "Hope."
    },
    {
        "question": "What do gun fanatics say about the fact that 130 studies prove gun control works? Why do they deny the facts?",
        "answer": ("As a gun fanatic, or a gun owner and Second Amendment supporter, one might say that gun control has not worked in cities like Chicago, New York, Los Angeles, Memphis, etc. "
                   "It appears that criminals still obtain guns. The belief is that criminals don’t care about the law, let alone gun control laws. Chicago has strict gun control measures but they "
                   "have not prevented gun crime. The real answer, according to this view, is to put criminals in prison and keep them there for a long time. Hence, the opposition to gun control is based "
                   "on the belief that it doesn't deter criminals.")
    },
    {
        "question": "Does your dog ever act offended by you?",
        "answer": ("Well, they tend to give you that look, lol. Many adorable expressions dogs make look like they are offended, worried, laughing, etc. In short, sometimes! If you're asking if a dog is "
                   "really offended, there’s no way of truly knowing, lol.")
    }
]

# Printing the organized data
for i, entry in enumerate(data, start=1):
    print(f"Entry {i}")
    print(f"Question: {entry['question']}")
    print(f"Answer: {entry['answer']}")
    print()


_df_0 = pd.DataFrame(data)

# Group by 'question' and calculate the size of each group
question_distribution = _df_0.groupby('question').size().reset_index(name='count')

# Print the distribution of questions
print("Question Distribution:")
print(question_distribution)

# Group by 'answer' and calculate the size of each group
answer_distribution = _df_0.groupby('answer').size().reset_index(name='count')

# Print the distribution of answers
print("\nAnswer Distribution:")
print(answer_distribution)


Entry 1
Question: Affordable web-based small business accounting software?
Answer: Hi, software simple & easy use. Profit-oriented accounting software. Analyze budget upcoming financial year. Also view past data within seconds. Free trial. Sign free trial: [GST Invoicing, Accounting & Filling Software](http://goo.gl/YA5QjJ) Best accounting software also easy use. Try! Thanks.

Entry 2
Question: Can Sweden enter NATO without Russia's consent?
Answer: Last knowledge update September 2021, Sweden is not a member of NATO (North Atlantic Treaty Organization). Therefore, it can enter NATO without Russia's consent. NATO is a political and military alliance of 30 member countries in North America and Europe. Countries seeking to join NATO must go through a process of accession, which involves negotiations, meeting certain criteria, and obtaining approval from existing NATO members. Sweden's foreign policy decisions are made by its government based on national interests, security considerations

In [36]:

# Printing the organized data
for i, entry in enumerate(data, start=1):
    print(f"Entry {i}")
    print(f"Question: {entry['question']}")
    print(f"Answer: {entry['answer']}")
    print()

Entry 1
Question: Affordable web-based small business accounting software?
Answer: Hi, software simple & easy use. Profit-oriented accounting software. Analyze budget upcoming financial year. Also view past data within seconds. Free trial. Sign free trial: [GST Invoicing, Accounting & Filling Software](http://goo.gl/YA5QjJ) Best accounting software also easy use. Try! Thanks.

Entry 2
Question: Can Sweden enter NATO without Russia's consent?
Answer: Last knowledge update September 2021, Sweden is not a member of NATO (North Atlantic Treaty Organization). Therefore, it can enter NATO without Russia's consent. NATO is a political and military alliance of 30 member countries in North America and Europe. Countries seeking to join NATO must go through a process of accession, which involves negotiations, meeting certain criteria, and obtaining approval from existing NATO members. Sweden's foreign policy decisions are made by its government based on national interests, security considerations

Tokenization

In [43]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')

df['question_tokens'] = df['question_cleaned'].apply(word_tokenize)
df['answer_tokens'] = df['answer_cleaned'].apply(word_tokenize)

print(df[['question_tokens', 'answer_tokens']])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


                                                                                                           question_tokens  \
0                                                            [affordable, webbased, small, business, accounting, software]   
1                                                                [right, sweden, enter, nato, without, russia, s, consent]   
2                                                                           [people, still, wear, speedo, swimsuit, beach]   
3                                           [gun, fanatic, say, fact, 130, study, proving, gun, control, work, deny, fact]   
4                                                                                       [dog, ever, act, offended, himher]   
...                                                                                                                    ...   
45116                                                                                           [hiking, gear, never, 

Remove Stop Words and Lemmitization

In [44]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_tokens(tokens):
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens

df['question_processed'] = df['question_tokens'].apply(preprocess_tokens)
df['answer_processed'] = df['answer_tokens'].apply(preprocess_tokens)

print(df[['question_processed', 'answer_processed']])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                                                                                        question_processed  \
0                                                            [affordable, webbased, small, business, accounting, software]   
1                                                                   [right, sweden, enter, nato, without, russia, consent]   
2                                                                           [people, still, wear, speedo, swimsuit, beach]   
3                                           [gun, fanatic, say, fact, 130, study, proving, gun, control, work, deny, fact]   
4                                                                                       [dog, ever, act, offended, himher]   
...                                                                                                                    ...   
45116                                                                                           [hiking, gear, never, 